<a href="https://colab.research.google.com/github/eduardodut/Analise-tweets-ICD-Unifor/blob/master/AnaliseTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Leitura dos dados

In [0]:
def downloadJsonFiles():
  # Fetch a single <1MB file using the raw GitHub URL.
  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/LulaOficial.json


  !curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://raw.githubusercontent.com/eduardodut/Analise-tweets-ICD-Unifor/master/Dados/jairbolsonaro.json     

downloadJsonFiles()

In [0]:

def carregarDados():
  import json
  import pandas as pd
  dataset_lula = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/LulaOficial.json') as lula_file:
    dados = json.load(lula_file)
    dataset_lula = pd.DataFrame(dados)


  dataset_bolso = pd.DataFrame()
  #tweetsLula = pd.read_json('/content/LulaOficial.json', lines = True)
  with open('/jairbolsonaro.json') as bolso_file:
    dados = json.load(bolso_file)
    dataset_bolso = pd.DataFrame(dados)

  return dataset_bolso, dataset_lula

dataset_bolso, dataset_lula = carregarDados()


In [97]:
dataset_bolso.loc[dataset_bolso['in_reply_to_status_id_str'] != 'None']

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,Thu Apr 01 02:59:50 +0000 2010,11402700435,11402700435,"EM ""DITADURA"" SEM PAREDÃO, ATÉ CHICO ALENCAR É...",False,"[0, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://about.twitter.com/products/tw...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,30,15,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Jan 10 11:02:23 +0000 2019,1083318129135112192,1083318129135112192,"Bom dia! 🇧🇷 #tbt com o amigo ""Canguru"", que já...",False,"[0, 90]","{'hashtags': [{'text': 'tbt', 'indices': [12, ...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,1773,31618,False,False,pt,"{'media': [{'id': 1083318121841262592, 'id_str...",False,NaN,NaN,NaN,NaN
2,Fri Jul 19 02:04:16 +0000 2019,1152036400138579968,1152036400138579968,- Para descontrair. Proibido queimar ovo. (Kkk...,False,"[0, 48]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,5859,46637,False,False,pt,"{'media': [{'id': 1152036204247953408, 'id_str...",False,NaN,NaN,NaN,NaN
3,Tue Oct 09 02:52:56 +0000 2018,1049492883328380928,1049492883328380928,Trecho de entrevista ao vivo para o Jornal Nac...,False,"[0, 65]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,13915,56322,False,False,pt,"{'media': [{'id': 1049489244387983360, 'id_str...",False,NaN,NaN,NaN,NaN
4,Wed Aug 23 15:12:48 +0000 2017,900375277557215232,900375277557215232,Querem criar o fundão bilionário na Reforma Po...,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,2271,7147,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6789,Tue Sep 17 16:39:08 +0000 2019,1173999841724325889,1173999841724325889,Menos burocratização e gastos ao brasileiro: a...,False,"[0, 280]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,6264,46305,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
6790,Sat Feb 20 12:32:40 +0000 2016,701021650167406592,701021650167406592,A @RevistaEpoca mente DESCARADAMENTE a meu res...,False,"[0, 98]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,452,893,False,False,pt,"{'media': [{'id': 701021612578103297, 'id_str'...",False,NaN,NaN,NaN,NaN
6791,Mon Jan 07 15:22:55 +0000 2019,1082296529434697728,1082296529434697728,Foi realizada hoje a Cerimônia de Posse dos Pr...,False,"[0, 279]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 128372940, 'id_str': '128372940', 'name...",None,None,None,None,False,5082,42443,False,False,pt,"{'media': [{'id': 1082296523990491137, 'id_str...",False,NaN,NaN,NaN,NaN
6792,Wed Oct 16 11:27:36 +0000 2019,11844306886086

In [92]:
dataset_lula.loc[dataset_lula['retweeted']==False]


,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,Fri Aug 17 16:23:04 +0000 2018,1030490201460801541,1030490201460801541,"""O presidente Lula vai dar um passeio nessa el...",False,"[0, 212]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,79,347,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
1,Tue Sep 16 01:53:02 +0000 2014,511694215735422976,511694215735422976,@dilmabr afirma que aqueles que querem acabar ...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,131574396.0,131574396,dilmabr,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,17,9,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
2,Tue Aug 08 23:23:49 +0000 2017,895063025996242944,895063025996242944,O PT deu cidadania para as mulheres fazerem po...,False,"[0, 95]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,72,203,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
3,Wed Oct 18 11:24:23 +0000 2017,920611514071076865,920611514071076865,Continuo sendo o lulinha paz e a amor. A gente...,False,"[0, 108]","{'hashtags': [{'text': 'LulanoRadio', 'indices...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,63,153,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
4,Mon Apr 24 22:24:18 +0000 2017,856634932764037120,856634932764037120,"Olhando as últimas pesquisas, vemos que a únic...",False,"[0, 100]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,63,168,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14956,Thu Oct 23 13:51:38 +0000 2014,525283408856219648,525283408856219648,O prefeito Eduardo Paes e @LindbergFarias acom...,False,"[0, 99]","{'hashtags': [{'text': 'QueroDilmaTreze', 'ind...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,19,8,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
14957,Tue Oct 02 13:23:07 +0000 2018,1047114757654884352,1047114757654884352,"Haddad, o candidato de Lula, com Lindberg sena...",False,"[0, 140]","{'hashtags': [{'text': 'HaddadP', 'indices': [...","<a href=""https://periscope.tv"" rel=""nofollow"">...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,52,191,False,False,pt,NaN,False,NaN,NaN,NaN,NaN
14958,Sun Jul 24 15:04:25 +0000 2016,757229960314617856,757229960314617856,"Nádia Campeão: ""não vamos permitir que essa ca...",False,"[0, 123]","{'hashtags': [{'text': 'SPcomHaddad', 'indices...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,None,"{'id': 2670726740, 'id_str': '2670726740', 'na...",None,None,None,None,False,5,12,False,False,pt,NaN,NaN,NaN,NaN,NaN,NaN
14959,Tue Jun 11 15:31:21 +0000 2019,1138468770232569860,1138468770232569860,"Os advogados de Lula, Cristiano Zanin e José R...",False,"[0, 232]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 267

In [67]:
print(dados.head().loc[0,1].keys())
print(type(dados.transpose()))
print(dados.transpose()[0][0].keys())

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])
<class 'pandas.core.frame.DataFrame'>
dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


In [47]:
 print(dados.transpose()[0][0])

{'created_at': 'Fri Aug 17 16:23:04 +0000 2018', 'id': 1030490201460801541, 'id_str': '1030490201460801541', 'full_text': '"O presidente Lula vai dar um passeio nessa eleição. Ganha no primeiro turno" - @Haddad_Fernando, vice na chapa, em coletiva de imprensa sobre a decisão da @ONUBrasil que assegura o direito de Lula ser candidato.', 'truncated': False, 'display_text_range': [0, 212], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Haddad_Fernando', 'name': 'Fernando Haddad', 'id': 354095556, 'id_str': '354095556', 'indices': [80, 96]}, {'screen_name': 'ONUBrasil', 'name': 'ONU Brasil', 'id': 57763444, 'id_str': '57763444', 'indices': [156, 166]}], 'urls': []}, 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2670726740, 'id_str': '2670726740', 'name

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
downloadJsonFiles()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     36      0 --:--:-- --:--:-- --:--:--    35
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    14  100    14    0     0     43      0 --:--:-- --:--:-- --:--:--    43
